<a href="https://colab.research.google.com/github/Thiago-Reis-Porto/treinamento-h2ia/blob/main/ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

Load the wine data

In [14]:
wine = load_wine()
data = pd.DataFrame(wine.data)
data.columns = wine.feature_names
target = pd.DataFrame(wine.target,columns=["target"])
data.insert(len(data.columns),"target",target) 

In [ ]:
data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


Split the data

In [15]:
train, test = train_test_split(data, test_size=0.3)
train.reset_index(inplace=True, drop=True); test.reset_index(inplace=True, drop=True)  
len(train), len(test)

(124, 54)

Train and show tree in preorder

In [16]:
model = ID3()
model.fit(train)
print("Preorder:\n")
model.print_tree(model.root, wine.target_names)

Preorder:

proline : left <=  755  right >  755
color_intensity : left <=  3.83  right >  3.83
class_1
flavanoids : left <=  1.69  right >  1.69
class_2
class_1
flavanoids : left <=  2.25  right >  2.25
malic_acid : left <=  2.32  right >  2.32
class_1
class_2
magnesium : left <=  134  right >  134
class_0
class_1


In [17]:
print("Error rate:")
print("{0:.0%}".format(model.validate(test)))

Error rate:
9%


In [18]:
print("K-Fold average error rate for k=10:")
k_folds(model, data)

K-Fold average error rate for k=10:
[0.05555555555555555, 0.1111111111111111, 0.05555555555555555, 0.1111111111111111, 0.05555555555555555, 0.0, 0.05555555555555555, 0.3888888888888889, 0.29411764705882354, 0.0]
Average error rate: 11%


In [19]:
print("Bootstrap average error rate for 10 splits and 20% test size:")
bootstrap(model, data)

Bootstrap average error rate for 10 splits and 20% test size:
[0.1111111111111111, 0.1111111111111111, 0.027777777777777776, 0.05555555555555555, 0.05555555555555555, 0.08333333333333333, 0.08333333333333333, 0.1388888888888889, 0.1388888888888889, 0.08333333333333333]
Average error rate: 9%


Load iris dataset

In [8]:
iris = load_iris()
data = pd.DataFrame(iris.data)
data.columns = iris.feature_names
target = pd.DataFrame(iris.target,columns=["target"])
data.insert(4,"target",target)

In [ ]:
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [9]:
train, test = train_test_split(data, test_size=0.3)
train.reset_index(inplace=True, drop=True); test.reset_index(inplace=True, drop=True)  
len(train), len(test)

(105, 45)

In [10]:
model = ID3()
model.fit(train)
print("Preorder:\n")
model.print_tree(model.root, iris.target_names)

Preorder:

petal length (cm) : left <=  2.45  right >  2.45
setosa
sepal length (cm) : left <=  5.75  right >  5.75
petal width (cm) : left <=  1.5  right >  1.5
versicolor
virginica
petal width (cm) : left <=  1.75  right >  1.75
versicolor
virginica


In [11]:
print("Error rate:")
print("{0:.0%}".format(model.validate(test)))

Error rate:
2%


In [12]:
print("K-Fold average error rate for k=10:")
k_folds(model, data)

K-Fold average error rate for k=10:
[0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.13333333333333333, 0.2, 0.0]
Average error rate: 5%


In [13]:
print("Bootstrap average error rate for 10 splits and 20% test size:")
bootstrap(model, data)

Bootstrap average error rate for 10 splits and 20% test size:
[0.03333333333333333, 0.03333333333333333, 0.0, 0.0, 0.06666666666666667, 0.03333333333333333, 0.03333333333333333, 0.0, 0.03333333333333333, 0.03333333333333333]
Average error rate: 3%


In [6]:
#-------------------------------------------------------------------------------
# ID3 decision tree
#-------------------------------------------------------------------------------
class ID3():
  def __init__(self):
    self.root = None
    self.features = None
#-------------------------------------------------------------------------------
  # Fits the data in the model and call gen tree for the root node
  def fit(self, data):
    features = features_cuts(data, data.columns.drop("target"))
  
    self.root = Node()
    self.gen_tree(self.root, data, features)
  
  # Generates a decision tree for the node
  def gen_tree(self, node, data, features):

      tgt = data.target.unique()
      # Returns classification if data has only one class
      if len(tgt) == 1: node.target = tgt[0]; return
      
      # No features
      if len(features) == 0:
        tgt = list(data.target)
        node.target = max(set(tgt), key = tgt.count)
        return
    
      feature = get_best_feature(data, features)
      features = features.copy()
      features = features_cuts(data, features)
      cut = features[feature]
      features.pop(feature)
      
      l_data = data[data[feature] <= cut]
      r_data = data[data[feature] > cut]
      
      # If one of childs would have no data
      if len(l_data) == 0 or len(r_data) == 0:
        tgt = list(data.target)
        node.target = max(set(tgt), key = tgt.count)
        return
        
      node.feature = feature
      node.cut = cut

      node.l_child = Node(parent=node)
      node.r_child = Node(parent=node)

      self.gen_tree(node.l_child, l_data, features)
      self.gen_tree(node.r_child, r_data, features)

      # If both childs have the same prediction
      pred_l = node.l_child.target
      pred_r = node.r_child.target
      if pred_l != None and pred_r != None:
        if pred_l == pred_r:
          node.target = pred_l
          node.feature = None
          node.cut = None
          node.l_child = None
          node.r_child = None
  
  # Validates with the test data
  def validate(self, data):
    errors = 0
    data_size = len(data)
    for i in data.index:#range(data_size):
      indiv = data.loc[i]
      pred = self.root.node_decision(indiv)
      if pred != indiv["target"]:
        errors+=1
    return errors/data_size
  
  # Return the data with predicted target
  def predict(self, data):
    for i in range(data_size):
      indiv = data.loc[i]
      pred = self.root.node_decision(indiv)
      indiv["target"] = pred

  # Prints the tree in pre order
  def print_tree(self, node, target_names):
    if node: 
      node.print_node(target_names)
      self.print_tree(node.l_child, target_names)
      self.print_tree(node.r_child, target_names)

    else: return 
#-------------------------------------------------------------------------------
# Node class with parent, feature, left/right children, target and cut
#-------------------------------------------------------------------------------
# Parent: point to parent node
# Feature: feauture of the node
# Cut: cut of the feature
# Target:if the node is a leaf target will contain the class decision.
#-------------------------------------------------------------------------------
class Node():
    def __init__(self, parent=None):
      self.parent = parent
      self.feature = None
      self.l_child = None
      self.r_child = None
      self.target = None
      self.cut = None
    
    # Method to take decision
    def node_decision(self, indiv):
      if self.feature:
        if indiv[self.feature] <= self.cut:
          return self.l_child.node_decision(indiv)
        else: return self.r_child.node_decision(indiv)
      else:
        return self.target
    # print the node, if its a leaf prints the predicted class
    def print_node(self, target_names):
      if self.feature:
        cut = '{0:.3g}'.format(self.cut)
        print(self.feature, ": left <= ", cut, " right > ", cut)
      else:
        print(target_names[self.target])
#-------------------------------------------------------------------------------

In [5]:
# Return the best feature for decision with the actual data
def get_best_feature(data, features):
  f = features.copy()
  for i in f:
    cut = f.get(i)
   #f[i] = gini_impurity(3, data, i, cut)
    f[i] = info_gain(3, data, i, cut)
  return max(f.items(), key=lambda x:x[1])[0]

# Returns a features dictionary with format: {feature: cut}
def features_cuts(data, f):
  c = get_cuts(data, f)
  features = {}
  for i, key  in enumerate(f): features[key] = c[i]
  return features

# Function to get the best features cuts (in half for decision)
def get_cuts(data, features):
  cuts = []
  for f in features:
    cut = []
    b = []
    unique =  data[f].sort_values().unique()
    for i, value in enumerate(unique):
      if value == unique[-1]: break
      cut.append((value + unique[i+1])/2)
      #b.append(gini_impurity(3, data, f, cut[i]))
      b.append(info_gain(3, data, f, cut[i]))
    cuts.append(cut[b.index(max(b))])
  return cuts


#not used
def gini_impurity(n_targets, dataset, feature, cut):
  gain = [0,0]
  size = [0,0]
  values = []
  sv = dataset[dataset[feature] <= cut]
  for i in range(2):
    gini_sum = 0
    size[i] = len(sv)
    if size[i] == 0: 
      sv = dataset[~dataset.isin(sv)].dropna()
      gain[i] = 1
      continue
    for j in range(n_targets):
      p = len(sv[sv["target"]==j])/size[i] 
      gini_sum += p**2
    gain[i] = 1 - gini_sum
    sv = dataset[~dataset.isin(sv)].dropna()
  size = [i/len(dataset) for i in size]
  return (size[0]*gain[0]) + (size[1]*gain[1])

In [3]:
def entropy(n_targets, dataset):
  p = []
  e_sum = 0
  zeros = 0
  for i in range(n_targets):
    p_i = len(dataset[dataset.target==i])/len(dataset)
    p.append(p_i)   
    if p_i == 0 : zeros += 1
  
  if zeros == n_targets - 1: return 0

  for i in p:
    if i != 0: e_sum -= i*np.log2(i)
  
  return e_sum

def info_gain(n_targets, dataset, feature, cut):
  gain_sum = 0
  sv = dataset[dataset[feature] <= cut]
  for i in range(2):
    sv_size = len(sv)
    if not sv_size:
      sv = dataset[dataset[feature] > cut]
      continue
    ev = entropy(n_targets, sv)
    pv = sv_size / len(dataset)
    gain_sum += pv*ev
    sv = dataset[dataset[feature] > cut]
  return entropy(n_targets, dataset) - gain_sum


In [4]:
def bootstrap(model, data, n_splits=10, test_size=.2):
  rs = ShuffleSplit(n_splits=n_splits, test_size=test_size)
  errors_rate = []
  
  for train_index, test_index in rs.split(data):
    x = data.loc[train_index]
    y = data.loc[test_index]
    model.fit(x)
    errors_rate.append(model.validate(y))

  print(errors_rate)
  print("Average error rate:", "{0:.0%}".format(sum(errors_rate) / len(errors_rate)))

In [2]:
def k_folds(model, data, n_splits=10):

  errors_rate = []
  kf = KFold(n_splits=n_splits)
  kf.get_n_splits(data)

  for train_index, test_index in kf.split(data):
    x, y = data.loc[train_index], data.loc[test_index]
    model.fit(x)
    errors_rate.append(model.validate(y))

  print(errors_rate)
  print("Average error rate:", "{0:.0%}".format(sum(errors_rate) / len(errors_rate)))